In [1]:
import os
import json
import mne
import scipy

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
from scipy.signal import hilbert, coherence

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline

mne.set_log_level('WARNING')

In [2]:
import my_functions as my_fun
import my_visualization_functions as my_vis_fun

In [ ]:
# Define subjects to process
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 2)]  
data_path = "eeg-motor-movementimagery-dataset-1.0.0/files/"

In [ ]:
# Process EEG data
eeg_data = my_fun.process_eeg(subjects=subjects, data_path=data_path, mode="automatic", apply_ica=True)

## Visualization for one subject

In [ ]:
subject = "S001"  # Change this to inspect other subjects
condition = "real_right_hand"  # Choose from real_left_hand, imagined_left_hand, etc.
epochs = eeg_data[subject][condition][0]

In [ ]:
my_vis_fun.plot_raw_eeg(eeg_data[subject][condition][0], subject, condition)

In [ ]:
my_vis_fun.plot_erd_ers(epochs, subject, condition)

## PLV and Coherence Pipeline for one subject

In [ ]:
real_epochs = mne.concatenate_epochs(eeg_data[subject]['real_right_hand'])
imagined_epochs = mne.concatenate_epochs(eeg_data[subject]['imagined_right_hand'])
rest_epochs = mne.concatenate_epochs(eeg_data[subject]['rest'])

In [ ]:
plv_real = my_fun.compute_plv_matrix(real_epochs)
plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
plv_rest = my_fun.compute_plv_matrix(rest_epochs)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real, real_epochs.ch_names, title="PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined, imagined_epochs.ch_names, title="PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_rest, rest_epochs.ch_names, title="PLV - Rest")

In [ ]:
my_vis_fun.plot_plv_difference(plv_real, plv_imagined, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real, plv_rest, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

In [ ]:
my_vis_fun.plot_motor_plv_difference(plv_real, plv_imagined, real_epochs.ch_names, title="PLV (Motor Cortex) - Real Right Hand")

## PLV and Coherence Pipeline for multiple subjects

In [3]:
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 10)]  

plv_results = {"real": [], "imagined": [], "rest": []}

eeg_data_all = {}  # <-- Store all subjects

for subject in subjects:
    print(f"\n🚀 Processing {subject}...")
    
    eeg_data = my_fun.process_eeg([subject], data_path="eeg-motor-movementimagery-dataset-1.0.0/files/", mode="automatic", apply_ica=False)

    # Merge subject's data into master dict
    eeg_data_all.update(eeg_data)

    # Compute PLV matrix (optional if you also want group matrix averaging)
    real_epochs = mne.concatenate_epochs(eeg_data[subject]["real_right_hand"])
    imagined_epochs = mne.concatenate_epochs(eeg_data[subject]["imagined_right_hand"])
    rest_epochs = mne.concatenate_epochs(eeg_data[subject]["rest"])

    plv_real = my_fun.compute_plv_matrix(real_epochs)
    plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
    plv_rest = my_fun.compute_plv_matrix(rest_epochs)

    plv_results["real"].append(plv_real)
    plv_results["imagined"].append(plv_imagined)
    plv_results["rest"].append(plv_rest)



🚀 Processing S001...

🔄 Processing S001...
✅ Extracted 7 epochs for S001 - R03 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R03
✅ Extracted 7 epochs for S001 - R07 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R07
✅ Extracted 8 epochs for S001 - R11 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R11
✅ Extracted 7 epochs for S001 - R04 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R04
✅ Extracted 7 epochs for S001 - R08 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R08
✅ Extracted 8 epochs for S001 - R12 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R12

✅ Processing complete for the subject.

🚀 Processing S002...

🔄 Processing S002...
✅ Extracted 7 epochs for S002 - R03 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R03
✅ Extracted 8 epochs for S002 - R07 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R07
✅ Extracted 7 epochs for S002 - R11 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - 

In [4]:
# Convert to numpy arrays for easy averaging
plv_real_group = np.mean(plv_results["real"], axis=0)
plv_imagined_group = np.mean(plv_results["imagined"], axis=0)
plv_rest_group = np.mean(plv_results["rest"], axis=0)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real_group, real_epochs.ch_names, title="Group Average PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Rest")

my_vis_fun.plot_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real_group, plv_rest_group, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

#my_vis_fun.plot_motor_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)


In [4]:
# Define subjects and channel pairs
#subjects = ["S001", "S002", "S005"]
conditions = {"real": "real_right_hand", "imagined": "imagined_right_hand", "rest": "rest"} 

channel_pairs = [
    ("C3", "C4"),  # Primary motor cortex
    ("C1", "C2"),  # Central midline
    ("FC3", "FC4"),  # Frontal-motor
    ("Cz", "CPz"),  # Midline execution
    ("CP3", "CP4"),  # Parietal-motor
    ("Fz", "Cz"),  # Frontal-central connection
    ("O1", "O2"),  # Occipital (unrelated)
    ("PO7", "PO8"),  # Parietal (unrelated)
    ("Fp1", "PO8")   # Prefrontal (unrelated)
]

# Analyze
df_plv_coh = my_fun.analyze_pairwise_plv_coherence(subjects, eeg_data_all, conditions, channel_pairs)

# Plot!
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="PLV Mean")
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="Coherence Mean")

🧠 Subject: S001
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S002
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S003
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S004
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S005
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S006
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S007
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S008
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest
🧠 Subject: S009
  ➡️ Condition: real
  ➡️ Condition: imagined
  ➡️ Condition: rest


In [5]:
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests

def paired_ttest_plv(df, metric="PLV Mean", alpha=0.05):
    pairs = df["Channel Pair"].unique()
    results = []

    # Step 1: Collect all p-values for both comparisons
    p_values_real_rest = []
    p_values_real_imagined = []

    for pair in pairs:
        df_pair = df[df["Channel Pair"] == pair]
        pivot_df = df_pair.pivot(index="Subject", columns="Condition", values=metric)
        pivot_df = pivot_df.dropna()

        t_real_rest, p_real_rest = ttest_rel(pivot_df["Real"], pivot_df["Rest"])
        t_real_imagined, p_real_imagined = ttest_rel(pivot_df["Real"], pivot_df["Imagined"])

        results.append({
            "Channel Pair": pair,
            "t Real vs Rest": t_real_rest,
            "p Real vs Rest": p_real_rest,
            "t Real vs Imagined": t_real_imagined,
            "p Real vs Imagined": p_real_imagined
        })

        p_values_real_rest.append(p_real_rest)
        p_values_real_imagined.append(p_real_imagined)

    # Step 2: Apply Bonferroni correction
    _, p_real_rest_corr, _, _ = multipletests(p_values_real_rest, alpha=alpha, method='bonferroni')
    _, p_real_imagined_corr, _, _ = multipletests(p_values_real_imagined, alpha=alpha, method='bonferroni')

    # Step 3: Merge corrected p-values back into results
    for i, res in enumerate(results):
        res["p Real vs Rest (Bonf)"] = p_real_rest_corr[i]
        res["p Real vs Imagined (Bonf)"] = p_real_imagined_corr[i]

    df_stats = pd.DataFrame(results)
    return df_stats


In [ ]:
from scipy.stats import ttest_rel

def paired_ttest_plv(df, metric="PLV Mean"):
    """
    Run paired t-tests for PLV between conditions on each channel pair.
    """
    pairs = df["Channel Pair"].unique()
    results = []

    for pair in pairs:
        df_pair = df[df["Channel Pair"] == pair]
        pivot_df = df_pair.pivot(index="Subject", columns="Condition", values=metric)
        pivot_df = pivot_df.dropna()

        t_real_rest, p_real_rest = ttest_rel(pivot_df["Real"], pivot_df["Rest"])
        t_real_imagined, p_real_imagined = ttest_rel(pivot_df["Real"], pivot_df["Imagined"])

        results.append({
            "Channel Pair": pair,
            "t Real vs Rest": t_real_rest,
            "p Real vs Rest": p_real_rest,
            "t Real vs Imagined": t_real_imagined,
            "p Real vs Imagined": p_real_imagined
        })

    df_stats = pd.DataFrame(results)
    return df_stats


In [6]:
def report_paired_ttests(df_stats):
    """
    Display t-test results for PLV in a clear and readable way.
    """
    for idx, row in df_stats.iterrows():
        print(f"\n📊 Channel Pair: {row['Channel Pair']}")

        # Real vs Rest
        p1 = row['p Real vs Rest']
        color1 = "\033[92m" if p1 <= 0.05 else "\033[90m"
        print(f"{color1}   Real vs Rest: t = {row['t Real vs Rest']:.2f}, p = {p1:.4f} \033[0m")

        # Real vs Imagined
        p2 = row['p Real vs Imagined']
        color2 = "\033[92m" if p2 <= 0.05 else "\033[90m"
        print(f"{color2}   Real vs Imagined: t = {row['t Real vs Imagined']:.2f}, p = {p2:.4f} \033[0m")


In [7]:
df_stats = paired_ttest_plv(df_plv_coh, metric="PLV Mean")
report_paired_ttests(df_stats)



📊 Channel Pair: C3-C4
   Real vs Rest: t = 10.47, p = 0.0000 
   Real vs Imagined: t = -0.81, p = 0.4401 

📊 Channel Pair: C1-C2
   Real vs Rest: t = 1.88, p = 0.0976 
   Real vs Imagined: t = -0.23, p = 0.8256 

📊 Channel Pair: FC3-FC4
   Real vs Rest: t = 1.88, p = 0.0971 
   Real vs Imagined: t = -0.84, p = 0.4272 

📊 Channel Pair: Cz-CPz
   Real vs Rest: t = 1.09, p = 0.3066 
   Real vs Imagined: t = 0.45, p = 0.6638 

📊 Channel Pair: CP3-CP4
   Real vs Rest: t = 7.95, p = 0.0000 
   Real vs Imagined: t = 0.02, p = 0.9852 

📊 Channel Pair: Fz-Cz
   Real vs Rest: t = 3.86, p = 0.0048 
   Real vs Imagined: t = -0.99, p = 0.3504 

📊 Channel Pair: O1-O2
   Real vs Rest: t = 0.37, p = 0.7230 
   Real vs Imagined: t = 0.29, p = 0.7811 

📊 Channel Pair: PO7-PO8
   Real vs Rest: t = 1.09, p = 0.3084 
   Real vs Imagined: t = 0.66, p = 0.5302 

📊 Channel Pair: Fp1-PO8
   Real vs Rest: t = 0.58, p = 0.5763 
   Real vs Imagined: t = -1.67, p = 0.1325 
